# Homework 3

**Name:** Omar Alejandro Guzmán Munguía

**e-mail:** omar.guzman5063@alumnos.udg.mx

# MODULES

In [2]:
# Load modules
import numpy as np
import math

import plotly.graph_objects as go

from scipy.stats import exponweib
from scipy.spatial import distance

import pandas as pd

# Activity 1: Path length - (BM1 vs BM2 vs CRW)

In [ ]:
# Function to be optimized


In [ ]:
# Run gradient descent algorithm

In [ ]:
# Plot the results